In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from env import Env
from agent import Agent

# Initialise Environment

In [ ]:
#initialise environment
workspace_lim = np.asarray([[-0.1 - 0.25, -0.1 + 0.25], 
                            [ 0.5 - 0.25,  0.5 + 0.25],
                            [         0.,        0.40]])
 
print(f"workspace_lim: {workspace_lim}")

obj_dir = 'objects/blocks/'
N_obj   = 1

env = Env(obj_dir = obj_dir, 
          N_obj   = N_obj,
          workspace_lim = workspace_lim)

# Initialise Agent

In [ ]:
agent = Agent(env)


# Testing

In [ ]:
color_img, depth_img = agent.env.get_rgbd_data()

In [ ]:
in_color_img, in_depth_img = agent.preprocess_input(color_img, depth_img)

In [ ]:
print(in_color_img.shape)
print(in_depth_img.shape)

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].imshow(in_depth_img.permute((1,2,0)))
ax[1].imshow(in_color_img.permute((1,2,0)))
plt.show()